In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Carregar dados
data = pd.read_csv('C:/Users/milen/OneDrive/Documentos/TCC/new/sem_clima.csv')

# Convertendo 'Data' para datetime e calculando a diferença diária do 'Preco_Dolar'
data['Data'] = pd.to_datetime(data['Data'])
data['Dif_Preco_Dolar'] = data['Preco_Dolar'].Dif()
data = data.dropna()

# Tratamento de outliers (exemplo simples)
# Substitua com seu método preferido de tratamento de outliers
# data = seu_metodo_de_tratamento_de_outliers(data)

# Separando os dados para treino, validação e teste
cutoff_date = pd.to_datetime("2023-01-01")
train_val_data = data[data['Data'] < cutoff_date]
test_data = data[data['Data'] >= cutoff_date]

# Preparação dos dados
features_train_val = train_val_data.drop(['Preco_Dolar', 'Dif_Preco_Dolar', 'Data'], axis=1)
target_train_val = train_val_data['Dif_Preco_Dolar']

# Dividindo treino e validação
features_train, features_val, target_train, target_val = train_test_split(features_train_val, target_train_val, test_size=0.2, random_state=42)

# Ajuste fino dos hiperparâmetros
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_features': ['auto', 'sqrt'],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'bootstrap': [True, False]
}

rf = RandomForestRegressor(random_state=42)
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(features_train, target_train)

best_params = grid_search.best_params_
print("Melhores hiperparâmetros:", best_params)

rf_best = RandomForestRegressor(**best_params, random_state=42)
rf_best.fit(features_train, target_train)

# Avaliação usando Gradient Boosting para comparação
gb = GradientBoostingRegressor(random_state=42)
gb.fit(features_train, target_train)

# Avaliação do modelo
window_size = 30
mse_values_rf, r2_values_rf = [], []
mse_values_gb, r2_values_gb = [], []
all_predictions_rf, all_predictions_gb = [], []
all_actual_values = []

for start in range(0, len(test_data) - window_size + 1):
    end = start + window_size
    test_subset = test_data.iloc[start:end]
    features_test = test_subset.drop(['Preco_Dolar', 'Dif_Preco_Dolar', 'Data'], axis=1)
    target_test = test_subset['Dif_Preco_Dolar']

    prediction_rf = rf_best.predict(features_test)
    prediction_gb = gb.predict(features_test)

    mse_values_rf.append(mean_squared_error(target_test, prediction_rf))
    r2_values_rf.append(r2_score(target_test, prediction_rf))

    mse_values_gb.append(mean_squared_error(target_test, prediction_gb))
    r2_values_gb.append(r2_score(target_test, prediction_gb))

    all_predictions_rf.extend(prediction_rf)
    all_predictions_gb.extend(prediction_gb)
    all_actual_values.extend(target_test)

# Exibindo os resultados para Random Forest
print("Random Forest - Média do MSE:", np.mean(mse_values_rf))
print("Random Forest - Média do R²:", np.mean(r2_values_rf))

# Exibindo os resultados para Gradient Boosting
print("Gradient Boosting - Média do MSE:", np.mean(mse_values_gb))
print("Gradient Boosting - Média do R²:", np.mean(r2_values_gb))


Fitting 3 folds for each of 144 candidates, totalling 432 fits


c:\Users\milen\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\ensemble\_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


Melhores hiperparâmetros: {'bootstrap': True, 'max_depth': 10, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200}


c:\Users\milen\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\ensemble\_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


Random Forest - Média do MSE: 9.146935874181615
Random Forest - Média do R²: -0.02339024755003896
Gradient Boosting - Média do MSE: 10.252676417009145
Gradient Boosting - Média do R²: -0.2633344220592247


In [26]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Carregar os dados
file_path_dolar =('C:/Users/milen/OneDrive/Documentos/TCC/new/sem_clima.csv')
data_dolar = pd.read_csv(file_path_dolar)
data_dolar.fillna(0, inplace=True)
# Converter a coluna 'Data' para datetime e ordenar os dados
data_dolar['Data'] = pd.to_datetime(data_dolar['Data'])
data_dolar = data_dolar.sort_values(by='Data')

# Calcular a diferença diária do 'Preco_Dolar'
data_dolar['Dif_Preco_Dolar'] = data_dolar['Preco_Dolar'].Dif()

# Remover a primeira linha
data_dolar = data_dolar.iloc[1:]

# Dividir os dados em treino, teste e validação
# Filtrar os dados para treino e teste
train_data = data_dolar[data_dolar['Data'] < pd.to_datetime("2020-01-01")]
test_data = data_dolar[data_dolar['Data'] >= pd.to_datetime("2020-01-01")]

# Selecionar variáveis dependentes e independentes
y_train = train_data['Dif_Preco_Dolar']
X_train = train_data.drop(columns=['Data', 'Preco_Dolar', 'Dif_Preco_Dolar'])

y_test = test_data['Dif_Preco_Dolar']
X_test = test_data.drop(columns=['Data', 'Preco_Dolar', 'Dif_Preco_Dolar'])

# O restante do código para tratamento de dados não numéricos, treinamento e avaliação do modelo permanece o mesmo.
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)
#Tratar valores não numéricos
non_numeric_columns = X_train.select_dtypes(include=['object']).columns
for col in non_numeric_columns:
    X_train[col] = pd.to_numeric(X_train[col].str.replace('.', '').str.replace(',', '.'), errors='coerce')
    X_test[col] = pd.to_numeric(X_test[col].str.replace('.', '').str.replace(',', '.'), errors='coerce')

# Treinar o modelo de regressão linear
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)
model = LinearRegression()

model.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred = model.predict(X_test)

# Calcular as métricas de desempenho
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Exibir os resultados
print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)

# Criar um DataFrame com as datas, valores reais e previstos
resultados = pd.DataFrame()
resultados['Data'] = test_data['Data']
resultados['Valor_Real'] = y_test
resultados['Valor_Previsto'] = y_pred

# Exibir os primeiros registros para verificar
print(resultados.head())

MSE: 9.595039206728583
MAE: 2.260820458170779
R^2: 0.18082554425127229
           Data  Valor_Real  Valor_Previsto
5524 2020-01-02       -3.67       -0.407749
5525 2020-01-03       -1.07       -0.872377
5526 2020-01-06       -5.45       -0.020286
5527 2020-01-07       -0.83        0.177764
5528 2020-01-08       -1.42        0.415451


C:\Users\milen\AppData\Local\Temp\ipykernel_24468\3619221553.py:38: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  X_train[col] = pd.to_numeric(X_train[col].str.replace('.', '').str.replace(',', '.'), errors='coerce')
C:\Users\milen\AppData\Local\Temp\ipykernel_24468\3619221553.py:39: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  X_test[col] = pd.to_numeric(X_test[col].str.replace('.', '').str.replace(',', '.'), errors='coerce')


In [89]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
from sklearn.preprocessing import StandardScaler


# Carregar os dados
file_path_dolar =('C:/Users/milen/OneDrive/Documentos/TCC/new/sem_clima_atualizado.csv')
data_dolar = pd.read_csv(file_path_dolar)
data_dolar.fillna(0, inplace=True)
# Converter a coluna 'Data' para datetime e ordenar os dados
data_dolar['Data'] = pd.to_datetime(data_dolar['Data'])
data_dolar = data_dolar.sort_values(by='Data')

# Calcular a diferença diária do 'Preco_Dolar'
#data_dolar['Dif_Preco_Dolar'] = data_dolar['Preco_Dolar'].Dif()

# Remover a primeira linha
data_dolar = data_dolar.iloc[1:]

# Dividir os dados em treino, teste e validação
# Filtrar os dados para treino e teste
train_data = data_dolar[data_dolar['Data'] < pd.to_datetime("2020-01-01")]
test_data = data_dolar[data_dolar['Data'] >= pd.to_datetime("2020-01-01")]

# Selecionar variáveis dependentes e independentes
y_train = train_data['Dif_Preco_Dolar']
X_train = train_data.drop(columns=['Data', 'Dif_Preco_Dolar'])

y_test = test_data['Dif_Preco_Dolar']
X_test = test_data.drop(columns=['Data', 'Dif_Preco_Dolar'])

# O restante do código para tratamento de dados não numéricos, treinamento e avaliação do modelo permanece o mesmo.
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)
#Tratar valores não numéricos
non_numeric_columns = X_train.select_dtypes(include=['object']).columns
for col in non_numeric_columns:
    X_train[col] = pd.to_numeric(X_train[col].str.replace('.', '').str.replace(',', '.'), errors='coerce')
    X_test[col] = pd.to_numeric(X_test[col].str.replace('.', '').str.replace(',', '.'), errors='coerce')

# Treinar o modelo de regressão linear
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)
model = LinearRegression()

model.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred = model.predict(X_test)

# Calcular as métricas de desempenho
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Exibir os resultados
print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)

# Criar um DataFrame com as datas, valores reais e previstos
resultados = pd.DataFrame()
resultados['Data'] = test_data['Data']
resultados['Valor_Real'] = y_test
resultados['Valor_Previsto'] = y_pred

# Exibir os primeiros registros para verificar
print(resultados.head())

MSE: 12.404340249641168
MAE: 2.6074201833883848
R^2: -0.05901794187522991
           Data  Valor_Real  Valor_Previsto
5524 2020-01-02       -3.67       -6.098756
5525 2020-01-03       -1.07       -1.238335
5526 2020-01-06       -5.45       -9.077853
5527 2020-01-07       -0.83       -2.184520
5528 2020-01-08       -1.42       -2.827011


In [147]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Carregar os dados
file_path_dolar =('C:/Users/milen/OneDrive/Documentos/TCC/new/sem_clima_cme.csv')
data_dolar = pd.read_csv(file_path_dolar)

# Converter a coluna 'Data' para datetime
data_dolar['Data'] = pd.to_datetime(data_dolar['Data'])

# Separar os últimos 10 dias de dados
ultimos_10_dias = data_dolar.tail(7)
data_dolar = data_dolar.iloc[:-7]

# Continuar com o pré-processamento como antes...

# Remover a primeira linha
data_dolar = data_dolar.iloc[1:]

# Função para remover outliers
def remove_outliers(df, column_names):
    for column in column_names:
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return df

# Aplicar a remoção de outliers
numeric_columns = data_dolar.select_dtypes(include=[np.number]).columns.tolist()
data_dolar = remove_outliers(data_dolar, numeric_columns)

# Dividir os dados em treino e teste
train_data = data_dolar[data_dolar['Data'] < pd.to_datetime("2020-01-01")]
test_data = data_dolar[data_dolar['Data'] >= pd.to_datetime("2020-01-01")]

# Selecionar variáveis dependentes e independentes
y_train = train_data['Dif_Preco_Dolar']
X_train = train_data.drop(columns=['Data','Dif_Preco_Dolar','Preco_Real', 'Dif_Preco_Real','Preco_Dolar'])

y_test = test_data['Dif_Preco_Dolar']
X_test = test_data.drop(columns=['Data','Dif_Preco_Dolar','Preco_Real', 'Dif_Preco_Real','Preco_Dolar'])

# Calcular a média para as colunas e substituir NaN pela média
mean_values_train = X_train.mean()
X_train.fillna(mean_values_train, inplace=True)

mean_values_test = X_test.mean()
X_test.fillna(mean_values_test, inplace=True)

# Tratar valores não numéricos
non_numeric_columns = X_train.select_dtypes(include=['object']).columns
for col in non_numeric_columns:
    X_train[col] = pd.to_numeric(X_train[col].str.replace('.', '').str.replace(',', '.'), errors='coerce')
    X_test[col] = pd.to_numeric(X_test[col].str.replace('.', '').str.replace(',', '.'), errors='coerce')

# Treinar o modelo de regressão linear
model = LinearRegression()
model.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred = model.predict(X_test)

# Calcular as métricas de desempenho
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Exibir os resultados
print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)

# Criar um DataFrame com as datas, valores reais e previstos
resultados = pd.DataFrame()
resultados['Data'] = test_data['Data']
resultados['Valor_Real'] = y_test
resultados['Valor_Previsto'] = y_pred

# Exibir os primeiros registros para verificar
print(resultados.head())

# Preparar dados dos últimos 10 dias para a previsão
X_ultimos_10_dias = ultimos_10_dias.drop(columns=['Data','Dif_Preco_Dolar','Preco_Real', 'Dif_Preco_Real','Preco_Dolar'])
X_ultimos_10_dias.fillna(mean_values_train, inplace=True)

# Prever os valores para 'ultimos_10_dias'
y_ultimos_10_dias_pred = model.predict(X_ultimos_10_dias)

# Comparar as previsões com os valores reais
ultimos_10_dias['Valor_Previsto'] = y_ultimos_10_dias_pred
print(ultimos_10_dias[['Data', 'Dif_Preco_Dolar', 'Valor_Previsto']])


MSE: 0.9983441660626964
MAE: 0.9991717400240543
R^2: nan
           Data  Valor_Real  Valor_Previsto
4231 2020-03-04       -1.95       -0.950828
           Data  Dif_Preco_Dolar  Valor_Previsto
5122 2023-10-30            -1.83       -0.352933
5123 2023-10-31             5.45        0.127794
5124 2023-11-01            -3.32        2.335962
5125 2023-11-03             7.86       39.668988
5126 2023-11-06             2.20        0.981174
5127 2023-11-07             0.43        0.755369
5128 2023-11-08             2.86       14.612960


R^2 score is not well-defined with less than two samples.


O MELHOR MELHOR

In [247]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import lightgbm as lgb
import shap

# Carregar os dados
file_path_dolar =('C:/Users/milen/OneDrive/Documentos/TCC/new/sem_clima_atualizado.csv')
data_dolar = pd.read_csv(file_path_dolar)

# Converter a coluna 'Data' para datetime e ordenar os dados
data_dolar['Data'] = pd.to_datetime(data_dolar['Data'])
# data_dolar = data_dolar.sort_values(by='Data')

# Remover a primeira linha
data_dolar = data_dolar.iloc[1:]

# Função para remover outliers
def remove_outliers(df, column_names):
    for column in column_names:
        Q1 = df[column].quantile(0.20)
        Q3 = df[column].quantile(0.80)
        IQR = Q3 - Q1
        lower_bound = Q1 - 0.4 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return df

# Aplicar a remoção de outliers
numeric_columns = data_dolar.select_dtypes(include=[np.number]).columns.tolist()
data_dolar = remove_outliers(data_dolar, numeric_columns)

# Dividir os dados em treino, teste e validação
train_data = data_dolar[data_dolar['Data'] < pd.to_datetime("2022-01-01")]
test_data = data_dolar[data_dolar['Data'] >= pd.to_datetime("2022-01-01")]

# Selecionar variáveis dependentes e independentes
y_train = train_data['Dif_Preco_Dolar']
X_train = train_data.drop(columns=['Data','Pontos', 'Dif_Preco_Dolar','TaxaSelic','OperacoesSeleic','UltimoEUR', 'Preco_Dolar', 'Preco_Real', 'Dif_Preco_Real'])

y_test = test_data['Dif_Preco_Dolar']
X_test = test_data.drop(columns=['Data','Pontos', 'Dif_Preco_Dolar','TaxaSelic','OperacoesSeleic','UltimoEUR', 'Preco_Dolar', 'Preco_Real', 'Dif_Preco_Real'])

# Calcular a média para as colunas e substituir NaN pela média
mean_values_train = X_train.mean()
X_train.fillna(mean_values_train, inplace=True)

mean_values_test = X_test.mean()
X_test.fillna(mean_values_test, inplace=True)

# Tratar valores não numéricos
non_numeric_columns = X_train.select_dtypes(include=['object']).columns
for col in non_numeric_columns:
    X_train[col] = pd.to_numeric(X_train[col].str.replace('.', '').str.replace(',', '.'), errors='coerce')
    X_test[col] = pd.to_numeric(X_test[col].str.replace('.', '').str.replace(',', '.'), errors='coerce')

# Treinar o modelo de regressão linear
model = LinearRegression()
#model = lgb.LGBMRegressor()
model.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred = model.predict(X_test)

# Calcular as métricas de desempenho
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Exibir os resultados
print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)

# Criar um DataFrame com as datas, valores reais e previstos
resultados = pd.DataFrame()
resultados['Data'] = test_data['Data']
resultados['Valor_Real'] = y_test
resultados['Valor_Previsto'] = y_pred

# Exibir os primeiros registros para verificar
print(resultados.head(20))


MSE: 0.9275394800923684
MAE: 0.8391132780534716
R^2: 0.3277792654327595
           Data  Valor_Real  Valor_Previsto
6391 2023-06-26        0.45        1.403968
6392 2023-06-27        0.10       -1.003845
6395 2023-06-30        1.30        2.252853
6396 2023-07-03       -0.63       -0.094444
6399 2023-07-06       -1.34       -2.022010
6404 2023-07-13        1.85        0.812661
6407 2023-07-18       -0.39        0.288551
6410 2023-07-21        2.79        1.164435
6411 2023-07-24        2.62        1.912676
6412 2023-07-25       -0.15       -1.257611
6413 2023-07-26        0.39        1.037881
6414 2023-07-27       -0.67        0.535776
6418 2023-08-02        0.03       -0.073454
6425 2023-08-11       -1.40        0.302492
6428 2023-08-16        0.19        0.523376
6430 2023-08-18        0.25       -0.571402
6431 2023-08-21        0.32       -0.066038
6432 2023-08-22        1.15        1.526829
6436 2023-08-28        1.29       -0.063466
6441 2023-09-04        0.78        0.829645


In [ ]:
# ... [seu código anterior permanece inalterado até o treinamento do modelo] ...

# Definindo o tamanho da janela deslizante
tamanho_da_janela = 5

# Inicializando o DataFrame de resultados
resultados = pd.DataFrame(columns=['Data', 'Valor_Real', 'Valor_Previsto'])

# Fazendo previsões em uma janela deslizante
for i in range(0, len(test_data), tamanho_da_janela):
    # Selecionar os dados da janela atual
    X_test_janela = X_test.iloc[i:i+tamanho_da_janela]
    y_test_janela = y_test.iloc[i:i+tamanho_da_janela]
    datas_janela = test_data['Data'].iloc[i:i+tamanho_da_janela]

    # Prever os valores para a janela atual
    y_pred_janela = model.predict(X_test_janela)

    # Armazenar os resultados
    janela_resultados = pd.DataFrame({
        'Data': datas_janela,
        'Valor_Real': y_test_janela,
        'Valor_Previsto': y_pred_janela
    })
    resultados = resultados.append(janela_resultados, ignore_index=True)

# Calcular as métricas de desempenho para todo o período de teste
mse = mean_squared_error(resultados['Valor_Real'], resultados['Valor_Previsto'])
mae = mean_absolute_error(resultados['Valor_Real'], resultados['Valor_Previsto'])
r2 = r2_score(resultados['Valor_Real'], resultados['Valor_Previsto'])

# Exibir os resultados
print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)
print(resultados.head(20))


In [269]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import lightgbm as lgb
import shap
import xgboost as xgb

# Carregar os dados
file_path_dolar =('C:/Users/milen/OneDrive/Documentos/TCC/new/sem_clima_atualizado.csv')
data_dolar = pd.read_csv(file_path_dolar)

# Converter a coluna 'Data' para datetime e ordenar os dados
data_dolar['Data'] = pd.to_datetime(data_dolar['Data'])
# data_dolar = data_dolar.sort_values(by='Data')

# Remover a primeira linha
data_dolar = data_dolar.iloc[1:]

# Função para remover outliers
def remove_outliers(df, column_names):
    for column in column_names:
        Q1 = df[column].quantile(0.20)
        Q3 = df[column].quantile(0.80)
        IQR = Q3 - Q1
        lower_bound = Q1 - 0.4 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return df

# Aplicar a remoção de outliers
numeric_columns = data_dolar.select_dtypes(include=[np.number]).columns.tolist()
data_dolar = remove_outliers(data_dolar, numeric_columns)

# Dividir os dados em treino, teste e validação
train_data = data_dolar[data_dolar['Data'] < pd.to_datetime("2020-01-01")]
test_data = data_dolar[data_dolar['Data'] >= pd.to_datetime("2020-01-01")]

# Selecionar variáveis dependentes e independentes
y_train = train_data['Dif_Preco_Dolar']
X_train = train_data.drop(columns=['Data', 'Pontos', 'Dif_Preco_Dolar', 'TaxaSelic','OperacoesSeleic','UltimoEUR', 'Preco_Dolar', 'Preco_Real','Dif_Preco_Real'])

y_test = test_data['Dif_Preco_Dolar']
X_test = test_data.drop(columns=['Data','Pontos', 'Dif_Preco_Dolar','TaxaSelic','OperacoesSeleic','UltimoEUR', 'Preco_Dolar', 'Preco_Real','Dif_Preco_Real'])

# Calcular a média para as colunas e substituir NaN pela média
mean_values_train = X_train.mean()
X_train.fillna(mean_values_train, inplace=True)

mean_values_test = X_test.mean()
X_test.fillna(mean_values_test, inplace=True)

# Tratar valores não numéricos
non_numeric_columns = X_train.select_dtypes(include=['object']).columns
for col in non_numeric_columns:
    X_train[col] = pd.to_numeric(X_train[col].str.replace('.', '').str.replace(',', '.'), errors='coerce')
    X_test[col] = pd.to_numeric(X_test[col].str.replace('.', '').str.replace(',', '.'), errors='coerce')

# Treinar o modelo de regressão linear
model = xgb.XGBRegressor(objective ='reg:squarederror')
#model = lgb.LGBMRegressor()
model.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred = model.predict(X_test)

# Calcular as métricas de desempenho
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Exibir os resultados
print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)

# Criar um DataFrame com as datas, valores reais e previstos
resultados = pd.DataFrame()
resultados['Data'] = test_data['Data']
resultados['Valor_Real'] = y_test
resultados['Valor_Previsto'] = y_pred

# Exibir os primeiros registros para verificar
print(resultados.head())


MSE: 3.1077767802435763
MAE: 1.4787053674773167
R^2: -1.2523159767585392
           Data  Valor_Real  Valor_Previsto
6391 2023-06-26        0.45        1.877941
6392 2023-06-27        0.10        1.564456
6395 2023-06-30        1.30        1.702488
6396 2023-07-03       -0.63        1.720908
6399 2023-07-06       -1.34        1.938566


In [185]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Carregar os dados
file_path_dolar = 'C:/Users/milen/OneDrive/Documentos/TCC/new/sem_clima_atualizado.csv'
data_dolar = pd.read_csv(file_path_dolar)

# Converter a coluna 'Data' para datetime e definir como índice
data_dolar['Data'] = pd.to_datetime(data_dolar['Data'])
data_dolar = data_dolar.set_index('Data')

# Identificar e tratar datas duplicadas (mantendo a primeira ocorrência)
data_dolar = data_dolar[~data_dolar.index.duplicated(keep='first')]

# Redefinir a frequência para diária
data_dolar = data_dolar.asfreq('D')

# Tratar valores faltantes (por exemplo, preenchendo com o valor anterior)
data_dolar = data_dolar.fillna(method='ffill')

# Selecionar a coluna de interesse para a análise ARIMA
y = data_dolar['Dif_Preco_Dolar']

# Dividir os dados em treino e teste
train_end = pd.to_datetime("2020-01-01")
y_train = y[y.index < train_end]
y_test = y[y.index >= train_end]

# Definir e ajustar o modelo ARIMA
p = 1  # Autoregressivo
d = 1  # Diferenciação
q = 1  # Média móvel
model = ARIMA(y_train, order=(p, d, q))
model_fit = model.fit()

# Fazer previsões
y_pred = model_fit.forecast(steps=len(y_test))

# Calcular métricas de desempenho
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Exibir resultados
print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)



MSE: 11.64856444442459
MAE: 2.5378110375356426
R^2: 6.189585840199463e-05


In [267]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Carregar os dados
file_path_dolar = 'C:/Users/milen/OneDrive/Documentos/TCC/new/sem_clima_atualizado.csv'
data_dolar = pd.read_csv(file_path_dolar)

# Converter a coluna 'Data' para datetime e ordenar os dados
data_dolar['Data'] = pd.to_datetime(data_dolar['Data'])
data_dolar = data_dolar.iloc[1:]

# Função para remover outliers
def remove_outliers(df, column_names):
    for column in column_names:
        Q1 = df[column].quantile(0.20)
        Q3 = df[column].quantile(0.80)
        IQR = Q3 - Q1
        lower_bound = Q1 - 0.4 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return df

# Aplicar a remoção de outliers
numeric_columns = data_dolar.select_dtypes(include=[np.number]).columns.tolist()
data_dolar = remove_outliers(data_dolar, numeric_columns)

# Dividir os dados em treino e teste
train_data = data_dolar[data_dolar['Data'] < pd.to_datetime("2020-01-01")]
test_data = data_dolar[data_dolar['Data'] >= pd.to_datetime("2020-01-01")]

# Selecionar variáveis dependentes e independentes
y_train = train_data['Dif_Preco_Dolar']
X_train = train_data.drop(columns=['Data', 'Pontos', 'Dif_Preco_Dolar', 'TaxaSelic', 'OperacoesSeleic', 'UltimoEUR', 'Preco_Dolar', 'Preco_Real', 'Dif_Preco_Real'])

y_test = test_data['Dif_Preco_Dolar']
X_test = test_data.drop(columns=['Data', 'Pontos', 'Dif_Preco_Dolar', 'TaxaSelic', 'OperacoesSeleic', 'UltimoEUR', 'Preco_Dolar', 'Preco_Real', 'Dif_Preco_Real'])

# Tratar valores não numéricos
non_numeric_columns = X_train.select_dtypes(include=['object']).columns
for col in non_numeric_columns:
    X_train[col] = pd.to_numeric(X_train[col].str.replace('.', '').str.replace(',', '.'), errors='coerce')
    X_test[col] = pd.to_numeric(X_test[col].str.replace('.', '').str.replace(',', '.'), errors='coerce')

# Normalizar os dados
scaler = MinMaxScaler(feature_range=(0, 1))
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Redimensionar os dados para o formato [samples, time_steps, features]
X_train_scaled = np.reshape(X_train_scaled, (X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_scaled = np.reshape(X_test_scaled, (X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

# Construir o modelo LSTM
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train_scaled.shape[1], X_train_scaled.shape[2])))
model.add(LSTM(units=50))
model.add(Dense(1))

# Compilar o modelo
model.compile(loss='mean_squared_error', optimizer='adam')

# Treinar o modelo
model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, verbose=2)

# Fazer previsões
y_pred = model.predict(X_test_scaled)

# Calcular as métricas de desempenho
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Exibir os resultados
print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)

# Criar um DataFrame com as datas, valores reais e previstos
resultados = pd.DataFrame()
resultados['Data'] = test_data['Data']
resultados['Valor_Real'] = y_test
resultados['Valor_Previsto'] = y_pred.flatten()  # Achatando a saída para combinar com o formato do y_test

# Exibir os primeiros registros para verificar
print(resultados.head())


Epoch 1/100
55/55 - 2s - loss: 3.4136 - 2s/epoch - 33ms/step
Epoch 2/100
55/55 - 0s - loss: 3.3149 - 101ms/epoch - 2ms/step
Epoch 3/100
55/55 - 0s - loss: 3.2934 - 100ms/epoch - 2ms/step
Epoch 4/100
55/55 - 0s - loss: 3.2817 - 96ms/epoch - 2ms/step
Epoch 5/100
55/55 - 0s - loss: 3.2835 - 96ms/epoch - 2ms/step
Epoch 6/100
55/55 - 0s - loss: 3.2842 - 95ms/epoch - 2ms/step
Epoch 7/100
55/55 - 0s - loss: 3.2744 - 97ms/epoch - 2ms/step
Epoch 8/100
55/55 - 0s - loss: 3.2678 - 98ms/epoch - 2ms/step
Epoch 9/100
55/55 - 0s - loss: 3.2653 - 114ms/epoch - 2ms/step
Epoch 10/100
55/55 - 0s - loss: 3.2640 - 101ms/epoch - 2ms/step
Epoch 11/100
55/55 - 0s - loss: 3.2640 - 113ms/epoch - 2ms/step
Epoch 12/100
55/55 - 0s - loss: 3.2613 - 120ms/epoch - 2ms/step
Epoch 13/100
55/55 - 0s - loss: 3.2659 - 102ms/epoch - 2ms/step
Epoch 14/100
55/55 - 0s - loss: 3.2572 - 101ms/epoch - 2ms/step
Epoch 15/100
55/55 - 0s - loss: 3.2644 - 98ms/epoch - 2ms/step
Epoch 16/100
55/55 - 0s - loss: 3.2523 - 98ms/epoch - 2ms

In [268]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Carregar os dados
file_path_dolar = 'C:/Users/milen/OneDrive/Documentos/TCC/new/sem_clima_atualizado.csv'
data_dolar = pd.read_csv(file_path_dolar)

# Converter a coluna 'Data' para datetime e ordenar os dados
data_dolar['Data'] = pd.to_datetime(data_dolar['Data'])

# Remover a primeira linha
data_dolar = data_dolar.iloc[1:]

# Função para remover outliers
def remove_outliers(df, column_names):
    for column in column_names:
        Q1 = df[column].quantile(0.20)
        Q3 = df[column].quantile(0.80)
        IQR = Q3 - Q1
        lower_bound = Q1 - 0.4 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return df

# Aplicar a remoção de outliers
numeric_columns = data_dolar.select_dtypes(include=[np.number]).columns.tolist()
data_dolar = remove_outliers(data_dolar, numeric_columns)

# Dividir os dados em treino e teste
train_data = data_dolar[data_dolar['Data'] < pd.to_datetime("2020-01-01")]
test_data = data_dolar[data_dolar['Data'] >= pd.to_datetime("2020-01-01")]

# Selecionar variáveis dependentes e independentes
y_train = train_data['Dif_Preco_Dolar']
X_train = train_data.drop(columns=['Data', 'Pontos', 'Dif_Preco_Dolar', 'TaxaSelic', 'OperacoesSeleic', 'UltimoEUR', 'Preco_Dolar', 'Preco_Real', 'Dif_Preco_Real'])

y_test = test_data['Dif_Preco_Dolar']
X_test = test_data.drop(columns=['Data', 'Pontos', 'Dif_Preco_Dolar', 'TaxaSelic', 'OperacoesSeleic', 'UltimoEUR', 'Preco_Dolar', 'Preco_Real', 'Dif_Preco_Real'])

# Calcular a média para as colunas e substituir NaN pela média
mean_values_train = X_train.mean()
X_train.fillna(mean_values_train, inplace=True)

mean_values_test = X_test.mean()
X_test.fillna(mean_values_test, inplace=True)

# Tratar valores não numéricos
non_numeric_columns = X_train.select_dtypes(include=['object']).columns
for col in non_numeric_columns:
    X_train[col] = pd.to_numeric(X_train[col].str.replace('.', '').str.replace(',', '.'), errors='coerce')
    X_test[col] = pd.to_numeric(X_test[col].str.replace('.', '').str.replace(',', '.'), errors='coerce')

# Treinar o modelo de regressão linear
model = LinearRegression()
model.fit(X_train, y_train)

# Definindo o tamanho da janela deslizante, por exemplo, 5 dias
tamanho_da_janela = 30

# Inicializando o DataFrame de resultados
resultados = pd.DataFrame()

# Fazendo previsões em uma janela deslizante
for i in range(0, len(test_data), tamanho_da_janela):
    # Selecionar os dados da janela atual
    X_test_janela = X_test.iloc[i:i+tamanho_da_janela]
    y_test_janela = y_test.iloc[i:i+tamanho_da_janela]
    datas_janela = test_data['Data'].iloc[i:i+tamanho_da_janela]

    # Prever os valores para a janela atual
    y_pred_janela = model.predict(X_test_janela)

    # Armazenar os resultados
    janela_resultados = pd.DataFrame({
        'Data': datas_janela,
        'Valor_Real': y_test_janela,
        'Valor_Previsto': y_pred_janela
    })
    resultados = pd.concat([resultados, janela_resultados], ignore_index=True)

# Calcular as métricas de desempenho para todo o período de teste
mse = mean_squared_error(resultados['Valor_Real'], resultados['Valor_Previsto'])
mae = mean_absolute_error(resultados['Valor_Real'], resultados['Valor_Previsto'])
r2 = r2_score(resultados['Valor_Real'], resultados['Valor_Previsto'])

# Exibir os resultados
print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)
print(resultados.head(20))


MSE: 0.9275394800923671
MAE: 0.839113278053472
R^2: 0.3277792654327605
         Data  Valor_Real  Valor_Previsto
0  2023-06-26        0.45        1.403968
1  2023-06-27        0.10       -1.003845
2  2023-06-30        1.30        2.252853
3  2023-07-03       -0.63       -0.094444
4  2023-07-06       -1.34       -2.022010
5  2023-07-13        1.85        0.812661
6  2023-07-18       -0.39        0.288551
7  2023-07-21        2.79        1.164435
8  2023-07-24        2.62        1.912676
9  2023-07-25       -0.15       -1.257611
10 2023-07-26        0.39        1.037881
11 2023-07-27       -0.67        0.535776
12 2023-08-02        0.03       -0.073454
13 2023-08-11       -1.40        0.302492
14 2023-08-16        0.19        0.523376
15 2023-08-18        0.25       -0.571402
16 2023-08-21        0.32       -0.066038
17 2023-08-22        1.15        1.526829
18 2023-08-28        1.29       -0.063466
19 2023-09-04        0.78        0.829645


In [273]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Carregar os dados
file_path_dolar = 'C:/Users/milen/OneDrive/Documentos/TCC/new/sem_clima_atualizado.csv'
data_dolar = pd.read_csv(file_path_dolar)

# Converter a coluna 'Data' para datetime
data_dolar['Data'] = pd.to_datetime(data_dolar['Data'])

# Remover a primeira linha
data_dolar = data_dolar.iloc[1:]

# Função para remover outliers
def remove_outliers(df, column_names):
    for column in column_names:
        Q1 = df[column].quantile(0.20)
        Q3 = df[column].quantile(0.80)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.2 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return df

# Aplicar a remoção de outliers
numeric_columns = data_dolar.select_dtypes(include=[np.number]).columns.tolist()
data_dolar = remove_outliers(data_dolar, numeric_columns)

# Dividir os dados em treino e teste
train_data = data_dolar[data_dolar['Data'] < pd.to_datetime("2020-01-01")]
test_data = data_dolar[data_dolar['Data'] >= pd.to_datetime("2020-01-01")]

# Selecionar variáveis dependentes e independentes
y_train = train_data['Dif_Preco_Dolar']
X_train = train_data.drop(columns=['Data', 'Pontos', 'Dif_Preco_Dolar', 'TaxaSelic', 'OperacoesSeleic', 'UltimoEUR', 'Preco_Dolar', 'Preco_Real', 'Dif_Preco_Real'])

y_test = test_data['Dif_Preco_Dolar']
X_test = test_data.drop(columns=['Data', 'Pontos', 'Dif_Preco_Dolar', 'TaxaSelic', 'OperacoesSeleic', 'UltimoEUR', 'Preco_Dolar', 'Preco_Real', 'Dif_Preco_Real'])

# Calcular a média para as colunas e substituir NaN pela média
mean_values_train = X_train.mean()
X_train.fillna(mean_values_train, inplace=True)

mean_values_test = X_test.mean()
X_test.fillna(mean_values_test, inplace=True)

# Tratar valores não numéricos
non_numeric_columns = X_train.select_dtypes(include=['object']).columns
for col in non_numeric_columns:
    X_train[col] = pd.to_numeric(X_train[col].str.replace('.', '').str.replace(',', '.'), errors='coerce')
    X_test[col] = pd.to_numeric(X_test[col].str.replace('.', '').str.replace(',', '.'), errors='coerce')

# Treinar o modelo de Random Forest
model = RandomForestRegressor()
model.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred = model.predict(X_test)

# Calcular as métricas de desempenho
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Exibir os resultados
print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)

# Criar um DataFrame com as datas, valores reais e previstos
resultados = pd.DataFrame()
resultados['Data'] = test_data['Data']
resultados['Valor_Real'] = y_test
resultados['Valor_Previsto'] = y_pred

# Exibir os primeiros registros para verificar
print(resultados.head())


MSE: 4.774370601507938
MAE: 1.8219976190476193
R^2: -0.8375899191562914
           Data  Valor_Real  Valor_Previsto
5527 2020-01-07       -0.83         -0.0092
5529 2020-01-09       -2.08          0.3886
5530 2020-01-10        1.16         -0.0817
5531 2020-01-13       -3.78          0.2513
5532 2020-01-14       -0.44          0.3201


In [275]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Carregar os dados
file_path_dolar = 'C:/Users/milen/OneDrive/Documentos/TCC/new/sem_clima_atualizado.csv'
data_dolar = pd.read_csv(file_path_dolar)

# Converter a coluna 'Data' para datetime
data_dolar['Data'] = pd.to_datetime(data_dolar['Data'])

# Remover a primeira linha
data_dolar = data_dolar.iloc[1:]

# Função para remover outliers
def remove_outliers(df, column_names):
    for column in column_names:
        Q1 = df[column].quantile(0.20)
        Q3 = df[column].quantile(0.80)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.2 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return df

# Aplicar a remoção de outliers
numeric_columns = data_dolar.select_dtypes(include=[np.number]).columns.tolist()
data_dolar = remove_outliers(data_dolar, numeric_columns)

# Dividir os dados em treino e teste
train_data = data_dolar[data_dolar['Data'] < pd.to_datetime("2020-01-01")]
test_data = data_dolar[data_dolar['Data'] >= pd.to_datetime("2020-01-01")]

# Selecionar variáveis dependentes e independentes
y_train = train_data['Dif_Preco_Dolar']
X_train = train_data.drop(columns=['Data', 'Pontos', 'Dif_Preco_Dolar', 'TaxaSelic', 'OperacoesSeleic', 'UltimoEUR', 'Preco_Dolar', 'Preco_Real', 'Dif_Preco_Real'])
y_test = test_data['Dif_Preco_Dolar']
X_test = test_data.drop(columns=['Data', 'Pontos', 'Dif_Preco_Dolar', 'TaxaSelic', 'OperacoesSeleic', 'UltimoEUR', 'Preco_Dolar', 'Preco_Real', 'Dif_Preco_Real'])

# Calcular a média para as colunas e substituir NaN pela média
mean_values_train = X_train.mean()
X_train.fillna(mean_values_train, inplace=True)
mean_values_test = X_test.mean()
X_test.fillna(mean_values_test, inplace=True)

# Tratar valores não numéricos
non_numeric_columns = X_train.select_dtypes(include=['object']).columns
for col in non_numeric_columns:
    X_train[col] = pd.to_numeric(X_train[col].str.replace('.', '').str.replace(',', '.'), errors='coerce')
    X_test[col] = pd.to_numeric(X_test[col].str.replace('.', '').str.replace(',', '.'), errors='coerce')

# Treinar o modelo de Random Forest
model = RandomForestRegressor()
model.fit(X_train, y_train)

# Definindo o tamanho da janela deslizante, por exemplo, 5 dias
tamanho_da_janela = 1

# Inicializando o DataFrame de resultados
resultados = pd.DataFrame()

# Fazendo previsões em uma janela deslizante
for i in range(0, len(test_data), tamanho_da_janela):
    # Selecionar os dados da janela atual
    X_test_janela = X_test.iloc[i:i+tamanho_da_janela]
    y_test_janela = y_test.iloc[i:i+tamanho_da_janela]
    datas_janela = test_data['Data'].iloc[i:i+tamanho_da_janela]

    # Prever os valores para a janela atual
    y_pred_janela = model.predict(X_test_janela)

    # Armazenar os resultados
    janela_resultados = pd.DataFrame({
        'Data': datas_janela,
        'Valor_Real': y_test_janela,
        'Valor_Previsto': y_pred_janela
    })
    resultados = pd.concat([resultados, janela_resultados], ignore_index=True)

# Calcular as métricas de desempenho para todo o período de teste
mse = mean_squared_error(resultados['Valor_Real'], resultados['Valor_Previsto'])
mae = mean_absolute_error(resultados['Valor_Real'], resultados['Valor_Previsto'])
r2 = r2_score(resultados['Valor_Real'], resultados['Valor_Previsto'])

# Exibir os resultados
print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)
print(resultados.head(20))


MSE: 3.5576828395238103
MAE: 1.5515880952380956
R^2: -0.36930344690867933
         Data  Valor_Real  Valor_Previsto
0  2020-01-07       -0.83          0.5165
1  2020-01-09       -2.08          0.1039
2  2020-01-10        1.16          0.1843
3  2020-01-13       -3.78          0.0860
4  2020-01-14       -0.44          0.2536
5  2020-01-15       -1.68         -0.0753
6  2020-01-16       -0.97         -0.2445
7  2020-01-17        1.16          0.0380
8  2020-01-20       -0.93         -0.3680
9  2020-01-21       -1.00         -0.2129
10 2020-01-22        1.19          0.0662
11 2020-01-23        0.52         -0.1438
12 2020-01-24       -1.34         -0.2713
13 2020-01-27       -2.92         -0.3079
14 2020-01-28        0.53         -0.0642
15 2020-01-29       -1.92         -0.4634
16 2020-01-30       -1.22         -0.2677
17 2020-01-31        0.28         -0.2239
18 2020-02-06       -0.19         -0.0974
19 2020-02-07       -0.42          0.0499


In [276]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Carregar os dados
file_path_dolar = 'C:/Users/milen/OneDrive/Documentos/TCC/new/sem_clima_atualizado.csv'
data_dolar = pd.read_csv(file_path_dolar)

# Converter a coluna 'Data' para datetime
data_dolar['Data'] = pd.to_datetime(data_dolar['Data'])

# Remover a primeira linha
data_dolar = data_dolar.iloc[1:]

# Função para remover outliers
def remove_outliers(df, column_names):
    for column in column_names:
        Q1 = df[column].quantile(0.20)
        Q3 = df[column].quantile(0.80)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.2 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return df

# Aplicar a remoção de outliers
numeric_columns = data_dolar.select_dtypes(include=[np.number]).columns.tolist()
data_dolar = remove_outliers(data_dolar, numeric_columns)

# Dividir os dados em treino e teste
train_data = data_dolar[data_dolar['Data'] < pd.to_datetime("2020-01-01")]
test_data = data_dolar[data_dolar['Data'] >= pd.to_datetime("2020-01-01")]

# Selecionar variáveis dependentes e independentes
y_train = train_data['Dif_Preco_Dolar']
X_train = train_data.drop(columns=['Data', 'Pontos', 'Dif_Preco_Dolar', 'TaxaSelic', 'OperacoesSeleic', 'UltimoEUR', 'Preco_Dolar', 'Preco_Real', 'Dif_Preco_Real'])
y_test = test_data['Dif_Preco_Dolar']
X_test = test_data.drop(columns=['Data', 'Pontos', 'Dif_Preco_Dolar', 'TaxaSelic', 'OperacoesSeleic', 'UltimoEUR', 'Preco_Dolar', 'Preco_Real', 'Dif_Preco_Real'])

# Calcular a média para as colunas e substituir NaN pela média
mean_values_train = X_train.mean()
X_train.fillna(mean_values_train, inplace=True)
mean_values_test = X_test.mean()
X_test.fillna(mean_values_test, inplace=True)

# Tratar valores não numéricos
non_numeric_columns = X_train.select_dtypes(include=['object']).columns
for col in non_numeric_columns:
    X_train[col] = pd.to_numeric(X_train[col].str.replace('.', '').str.replace(',', '.'), errors='coerce')
    X_test[col] = pd.to_numeric(X_test[col].str.replace('.', '').str.replace(',', '.'), errors='coerce')

# Treinar o modelo de Random Forest
model = RandomForestRegressor()
model.fit(X_train, y_train)

# Definindo o tamanho da janela deslizante, por exemplo, 5 dias
tamanho_da_janela = 30

# Inicializando o DataFrame de resultados
resultados = pd.DataFrame()

# Fazendo previsões em uma janela deslizante
for i in range(0, len(test_data), tamanho_da_janela):
    # Selecionar os dados da janela atual
    X_test_janela = X_test.iloc[i:i+tamanho_da_janela]
    y_test_janela = y_test.iloc[i:i+tamanho_da_janela]
    datas_janela = test_data['Data'].iloc[i:i+tamanho_da_janela]

    # Prever os valores para a janela atual
    y_pred_janela = model.predict(X_test_janela)

    # Armazenar os resultados
    janela_resultados = pd.DataFrame({
        'Data': datas_janela,
        'Valor_Real': y_test_janela,
        'Valor_Previsto': y_pred_janela
    })
    resultados = pd.concat([resultados, janela_resultados], ignore_index=True)

# Calcular as métricas de desempenho para todo o período de teste
mse = mean_squared_error(resultados['Valor_Real'], resultados['Valor_Previsto'])
mae = mean_absolute_error(resultados['Valor_Real'], resultados['Valor_Previsto'])
r2 = r2_score(resultados['Valor_Real'], resultados['Valor_Previsto'])

# Exibir os resultados
print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)
print(resultados.head(20))


MSE: 3.9570341473015884
MAE: 1.6424206349206354
R^2: -0.523008301144869
         Data  Valor_Real  Valor_Previsto
0  2020-01-07       -0.83          0.1661
1  2020-01-09       -2.08          0.0009
2  2020-01-10        1.16         -0.0827
3  2020-01-13       -3.78         -0.2738
4  2020-01-14       -0.44          0.0804
5  2020-01-15       -1.68         -0.1533
6  2020-01-16       -0.97         -0.2727
7  2020-01-17        1.16         -0.0938
8  2020-01-20       -0.93         -0.4045
9  2020-01-21       -1.00         -0.3433
10 2020-01-22        1.19         -0.0695
11 2020-01-23        0.52         -0.2620
12 2020-01-24       -1.34         -0.3953
13 2020-01-27       -2.92         -0.3882
14 2020-01-28        0.53          0.1923
15 2020-01-29       -1.92         -0.4514
16 2020-01-30       -1.22         -0.2726
17 2020-01-31        0.28         -0.3527
18 2020-02-06       -0.19         -0.2504
19 2020-02-07       -0.42         -0.0880
